# Description

It is a model for creating a demonstration of a model using jupyter-dash. Random data was generated using sklearn library. The model itself predicts the housing prices in Tashkent based on various features. Random forest regressor was used as a model.  







# Data Preparation

In [ ]:
### imports ###

import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

In [ ]:
### generate pseudo-data for model ###

X, y = make_regression(n_samples=10000, n_features=10, noise=0.1)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
### process data for training purposes ###

MIN_PRICE = 10000
PRICE_SCALE = 100000

# normalize input data
min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler.fit(x_train)
x_train = min_max_scaler.transform(x_train)
x_test = min_max_scaler.transform(x_test)
min_max_scaler.fit(y_train.reshape(-1, 1))
y_train = min_max_scaler.transform(y_train.reshape(-1, 1))
y_test = min_max_scaler.transform(y_test.reshape(-1, 1))

y_train = (PRICE_SCALE * y_train) + MIN_PRICE
y_test = (PRICE_SCALE * y_test) + MIN_PRICE

# translating to pandas for usage ease
df_x_train = pd.DataFrame(x_train)
df_y_train = pd.DataFrame(y_train)
df_x_test = pd.DataFrame(x_test)
df_y_test = pd.DataFrame(y_test)

# naming the columns
df_x_train.columns = ['rooms', 'center', 'parks', 'malls', 'security', 'transport', 'schools', 'parking', 'highway', 'restaraunts']
df_x_test.columns = ['rooms', 'center', 'parks', 'malls', 'security', 'transport', 'schools', 'parking', 'highway', 'restaraunts']
df_y_train.columns = ['price']
df_y_test.columns = ['price']

# Model

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
model = RandomForestRegressor(n_estimators=500, min_samples_split=2, random_state=42).fit(df_x_train, df_y_train)
y_pred = model.predict(df_x_test)
mean_absolute_percentage_error(y_test, y_pred)

# Model Demo

In [ ]:
# install jupyter_dash library if needed
!pip install jupyter_dash

In [ ]:
### imports ###

from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [ ]:
JupyterDash.infer_jupyter_proxy_config()

In [ ]:
### Demo of a service model ###

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)


def create_slider(slider_id):
  levels = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
  return dcc.Slider(
          id=slider_id,
          min = 1,
          max = 10,
          marks = {str(level): str(level) for level in levels},
          value = 4)
  

app.layout = html.Div([
  html.H3('Demo of a housing price estimator in Tashkent, Uzbekistan', style={'text-align': 'center'}),
  html.Br(),

  html.Label('Number of rooms', style={'text-align': 'center'}),
  create_slider('rooms'),
  html.Br(),

  html.Label('Distance from center', style={'text-align': 'center'}),
  create_slider('center'),
  html.Br(),

  html.Label('Closeness to malls', style={'text-align': 'center'}),
  create_slider('malls'),
  html.Br(),            

  html.Label('Parking space', style={'text-align': 'center'}),
  create_slider('parking'),
  html.Br(), 
  
  html.Label('Safety', style={'text-align': 'center'}),
  create_slider('safety'),
  html.Br(),

  html.Label('Transportation availability', style={'text-align': 'center'}),
  create_slider('transport'),
  html.Br(),            

  html.Label('Schools availability', style={'text-align': 'center'}),
  create_slider('schools'), 
  html.Br(),

  html.Label('Restaurants availability', style={'text-align': 'center'}),
  create_slider('restaurants'),
  html.Br(),

  html.Label('Closeness to highways', style={'text-align': 'center'}),
  create_slider('highway'),
  html.Br(),
    
  html.Br(),
  html.H4(id='price', style={'text-align': 'center'}),
])

@app.callback(
  Output('price', 'children'),
  Input('rooms', 'value'),
  Input('center', 'value'),
  Input('malls', 'value'),
  Input('parking', 'value'),
  Input('safety', 'value'),
  Input('transport', 'value'),
  Input('schools', 'value'),
  Input('restaurants', 'value'),
  Input('highway', 'value'),
)
def update_output_div(rooms, center, malls, parking, safety, 
                      transport, schools, restaurants, highway):
    rooms = float(rooms) / 10
    center = float(center) / 10
    parks = 0.5
    malls = float(malls) / 10
    parking = float(parking) / 10
    safety = float(safety) / 10
    transport = float(transport) / 10
    schools = float(schools) / 10
    highway = float(highway) / 10
    restaurants = float(restaurants) / 10
    x_input = pd.DataFrame([[rooms, center, parks, malls, parking, safety, transport, schools, highway, restaurants]], columns = list(df_x_train.columns))
    y_output = model.predict(x_input)
    return 'Estimated Price: {}'.format(round(y_output[0]))

In [ ]:
# run the demo
app.run_server()